# Phase 1. Subphase 1.

Mario Chacón Falcón & Jose M. Moyano.

The goal of this first experiment is to decide which parameter for public_data_prediction works better, majoriy_voting or weighted_majority_voting. 

In order to do so, the rest of the model is fixed to its simplest version, which is:

server_alpha_weight_adj = 'own'

prediction_weights = 'server_and_clients'

balanced_target_client_weights = False

adapt_client_weights = np.ones(Nclients)

Nclients=10

T (iterations) = 10

To test it out, four datasets are selected, adult dataset (unbalanced), nursery dataset (more than 2 labels), bank and breast cancer.

The test will be done with 6 different seeds, creating each time 5folds, and hence having 30 tests. The mean over the accuracy and f1score of the FL model of all clients is calculated and stored in a dataframe for further statistics tests. 

In [1]:
from flextrees.datasets.tabular_datasets import bank
from flextrees.datasets.tabular_datasets import nursery
from flextrees.datasets.tabular_datasets import adult
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from utils.evaluation import FL_simple_cross_val_score
from sklearn.datasets import load_breast_cancer
import time
from statds.no_parametrics import wilconxon

Let's first load the data

In [2]:
train_data, test_data = bank(ret_feature_names=False, categorical=False)
X_data,y_data = train_data.to_numpy()
X_test,y_test = test_data.to_numpy()
X_data_bank = np.concatenate((X_data,X_test)).astype(int) #Some were bools
targets_bank = np.concatenate((y_data,y_test))
X_data_bank.shape

(45211, 31)

In [3]:
train_data, test_data = adult(ret_feature_names=False, categorical=False)
X_data,y_data = train_data.to_numpy()
X_test,y_test = test_data.to_numpy()
X_data_adult = np.concatenate((X_data,X_test))
targets_adult = np.concatenate((y_data,y_test))
X_data_adult.shape

(32561, 108)

In [4]:
train_data, test_data = nursery(ret_feature_names=False, categorical=False)
X_data,y_data = train_data.to_numpy()
X_test,y_test = test_data.to_numpy()
X_data_nursery = np.concatenate((X_data,X_test))
targets_nursery = np.concatenate((y_data,y_test))
X_data_nursery.shape

(12960, 27)

In [5]:
breast_cancer = load_breast_cancer()
X_data_bcancer = breast_cancer.data
targets_bcancer = breast_cancer.target

In [6]:
#seeds =[0,142,242,342,442,542]
#parameters = ['majority_voting','weighted_majority_voting']
#datasets = {'bank':(X_data_bank,targets_bank),
#            'adult':(X_data_adult,targets_adult),'nursery':(X_data_nursery,targets_nursery)}
#parameterscolumns = ['majority_voting','maj_vot_std','weighted_majority_voting','w_m_v_std']

In [7]:
def experiment1search(datasets,data_distribution,distribution_param = None):

    #Algorithm parameters:
    Nclients = 10
    server_alpha_weight_adj = 'own' 
    prediction_weights = 'server_and_clients'
    T = 10
    adapt_client_weight = np.ones(Nclients)
    parameters = ['majority_voting','weighted_majority_voting']

    #Default settings
    seeds =[0,142,242,342,442,542]
    parameterscolumns = ['majority_voting','maj_vot_std','weighted_majority_voting','w_m_v_std']

    #Creates dataframes to store avg results : FL on global data, FL on local data, centralized model scores (f1 and acc)
    dataframeacc_globaldata = pd.DataFrame(np.nan, index= datasets.keys(),
                            columns = parameterscolumns)
    dataframeacc_globaldata = dataframeacc_globaldata.rename_axis('FLGlobal_acc',axis=1)
    dataframef1_globaldata = pd.DataFrame(np.nan, index= datasets.keys(),
                            columns = parameterscolumns)
    dataframef1_globaldata = dataframef1_globaldata.rename_axis('FLGlobal_f1',axis=1)
    dataframeacc_localdata = pd.DataFrame(np.nan, index= datasets.keys(),
                            columns = parameterscolumns)
    dataframeacc_localdata = dataframeacc_localdata.rename_axis('FLlocal_acc',axis=1)
    dataframef1_localdata = pd.DataFrame(np.nan, index= datasets.keys(),
                            columns = parameterscolumns)
    dataframef1_localdata = dataframef1_localdata.rename_axis('FLlocal_f1',axis=1)
    dataframe_centralizedacc = pd.DataFrame(np.nan, index = datasets.keys(),columns = ['mean','std'])
    dataframe_centralizedacc = dataframe_centralizedacc.rename_axis('centralized_acc',axis=1)
    dataframe_centralizedf1 = pd.DataFrame(np.nan, index = datasets.keys(),columns = ['mean','std'])
    dataframe_centralizedf1 = dataframe_centralizedf1.rename_axis('centralized_f1',axis=1)

    #Store each result for statistics tests
    acc_results = []
    f1_results = []

    #Training of 6 5-folds for each dataset and then gets stored in its corresponding row. 
    for dataset,data in datasets.items():
        data_to_mergeacc = [[]]*len(parameters)
        data_to_mergef1 = [[]]*len(parameters)
        centralized_acc = []
        centralized_f1 = []
        for seed in seeds:
            fulldata,fulltargets = data
            X_data,public_data,y_data,public_data_targets = train_test_split(fulldata,fulltargets,
                                                                            train_size=0.75,random_state=seed)
            #Create folds
            skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=seed)
            indices_iterable = skf.split(X_data,y_data)
            
            dataset_splits = [None]*5
            for i,(filter_train,filter_test) in enumerate(indices_iterable):
                dataset_splits[i] = (X_data[filter_train],X_data[filter_test],
                                    y_data[filter_train],y_data[filter_test])
                
            aux_acc_dataframe = pd.DataFrame(np.nan, index = [dataset+str(seed)]*len(dataset_splits),columns = parameters)
            aux_f1_dataframe = pd.DataFrame(np.nan, index = [dataset+str(seed)]*len(dataset_splits),columns = parameters)   
            for i,parameter in enumerate(parameters):
                
                FLmodels,centralized_models,accDataFrames,F1DataFrames,acc_centralized_model_scores,f1_centralized_model_scores = FL_simple_cross_val_score(dataset_splits,
                                public_data,Nclients=Nclients,
                                public_data_prediction=parameter,server_alpha_weight_adj=server_alpha_weight_adj,
                                prediction_weights=prediction_weights,T=T,data_distribution=data_distribution,distribution_param=distribution_param,random_state=seed,
                                adapt_client_weight=adapt_client_weight)
                for j,accDataFrame in enumerate(accDataFrames):
                    aux_acc_dataframe.iloc[j,i] = accDataFrame.loc[:]['FL_acc_global_data'].mean()
                    aux_f1_dataframe.iloc[j,i] = F1DataFrames[j].loc[:]['FL_acc_global_data'].mean()
                
                data_to_mergeacc[i] = data_to_mergeacc[i] + accDataFrames
                data_to_mergef1[i] = data_to_mergef1[i] + F1DataFrames

            #Stores the dataframe of the results of the 5-fold in each parameter to merge them all
            acc_results.append(aux_acc_dataframe)
            f1_results.append(aux_f1_dataframe)

            #In each parameter the centralized score does not change, hence, we only store the last ones.
            centralized_acc = centralized_acc + acc_centralized_model_scores
            centralized_f1 = centralized_f1 + f1_centralized_model_scores
        #Once we've done all the tests in all the parameters and seeds for the given dataset, we store it in the dataframes and 
        #start with the next one
        for i in range(len(data_to_mergeacc)):
            data_to_mergeacc[i] = pd.concat(data_to_mergeacc[i],axis=0).describe().loc[['mean','std']][['FL_acc_own_data','FL_acc_global_data']]
            data_to_mergef1[i] = pd.concat(data_to_mergef1[i],axis=0).describe().loc[['mean','std']][['FL_acc_own_data','FL_acc_global_data']]
        
        arr_centralized_acc = np.array(centralized_acc)
        arr_centralized_f1 = np.array(centralized_f1)
        dataframeacc = pd.concat(data_to_mergeacc,axis=0)
        dataframef1 = pd.concat(data_to_mergef1,axis=0)
        dataframeacc_globaldata.loc[dataset] = dataframeacc.iloc[:,1].values
        dataframeacc_localdata.loc[dataset] = dataframeacc.iloc[:,0].values
        dataframef1_globaldata.loc[dataset] = dataframef1.iloc[:,1].values
        dataframef1_localdata.loc[dataset] = dataframef1.iloc[:,0].values
        dataframe_centralizedacc.loc[dataset] = (arr_centralized_acc.mean(),arr_centralized_acc.std())
        dataframe_centralizedf1.loc[dataset] = (arr_centralized_f1.mean(),arr_centralized_f1.std())
    
    statistics_acc = pd.concat(acc_results,axis=0)
    statistics_acc = statistics_acc.rename_axis('FL_global_acc')
    statistics_f1 = pd.concat(f1_results,axis=0)
    statistics_f1 = statistics_f1.rename_axis('FL_global_f1')

    return dataframeacc_globaldata,dataframeacc_localdata,dataframe_centralizedacc,dataframef1_globaldata,dataframef1_localdata,dataframe_centralizedf1,statistics_acc,statistics_f1



In [20]:
import math

#Código copiado de la libreria tomando r_plus y r_minus y devolviendolo(No lo hago todo porque el wilconxon de la librería 
#devuelve también el critical value con una función auxiliar (que tiene sentido si el sample es <25(que no es nuestro caso)
# y devuelve el p_valor de la normal (con una función auxiliar) asociado al estadístico (que tiene sentido si el sample es mayor que 25))

def wilconxonbest(dataset: pd.DataFrame, alpha: float = 0.05, verbose: bool = False):
    """
    Perform the Wilcoxon signed-rank test. This non-parametric test is used to compare two related samples, matched
    samples, or repeated measurements on a single sample to assess whether their population mean ranks differ. It is
    an alternative to the paired Student's t-test when the data is not normally distributed.

    Parameters
    ----------
    dataset : pandas.DataFrame
        A DataFrame with exactly two columns, each representing a different condition or time point for the same
        subjects.
    alpha : float, optional
        The significance level for the test. Default is 0.05.
    verbose : bool, optional
        If True, prints the detailed results table.

    Returns
    -------
    w_wilcoxon : float
        The Wilcoxon test statistic, which is the smallest of the sums of the positive and negative ranks.
    p_value : float or None
        The p-value for the hypothesis test (only for large sample sizes, otherwise None).
    cv_alpha_selected : float or None
        The critical value for the test at the specified alpha level (only for small sample sizes, otherwise None).
    hypothesis : str
        A string stating the conclusion of the test based on the test statistic, critical value, or p-value and alpha.

    Note
    ----
    The Wilcoxon signed-rank test makes fewer assumptions than the t-test and is appropriate when the data
    are not normally distributed. It ranks the absolute differences between pairs, then compares these ranks.
    The test is sensitive to ties and has different procedures for small and large sample sizes. For large samples,
    the test statistic is approximately normally distributed, allowing the use of normal approximation for p-value
    calculation.
    """
    if dataset.shape[1] != 2:
        raise "Error: The test only needs two samples"

    results_table = dataset.copy()
    columns = list(dataset.columns)
    differences_results = dataset[columns[0]] - dataset[columns[1]]
    absolute_dif = differences_results.abs()
    absolute_dif = absolute_dif.sort_values()
    results_wilconxon = {"index": [], "dif": [], "rank": [], "R": []}
    rank = 0.0
    tied_ranges = not (len(set(absolute_dif)) == absolute_dif.shape[0])
    for index in absolute_dif.index:
        if math.fabs(0 - absolute_dif[index]) < 1e-10:
            continue
        rank += 1.0
        results_wilconxon["index"] += [index]
        results_wilconxon["dif"] += [differences_results[index]]
        results_wilconxon["rank"] += [rank]
        results_wilconxon["R"] += ["+" if differences_results[index] > 0 else "-"]

    df = pd.DataFrame(results_wilconxon)
    df = df.set_index("index")
    df = df.sort_index()
    results_table = pd.concat([results_table, df], axis=1)

    tie_sum = 0

    if tied_ranges:
        vector = [abs(i) for i in results_table["dif"]]

        counts = {}
        for number in vector:
            try:
                counts[number] = counts[number] + 1
            except KeyError:
                counts[number] = 1

        ranks = results_table["rank"].to_numpy()
        for index, number in enumerate(vector):
            if counts[number] > 1:
                rank_sum = sum(ranks[i] for i, x in enumerate(vector) if x == number)
                average_rank = rank_sum / counts[number]
                for i, x in enumerate(vector):
                    if x == number:
                        ranks[i] = average_rank
        tie_sizes = np.array(list(counts.values()))
        tie_sum = (tie_sizes ** 3 - tie_sizes).sum()

    if verbose:
        print(results_table)

    r_plus = results_table[results_table.R == "+"]["rank"].sum()
    r_minus = results_table[results_table.R == "-"]["rank"].sum()

    #w_wilcoxon = min([r_plus, r_minus])
    #num_problems = results_table.shape[0] - (results_table.R.isna().sum())
    #mean_wilcoxon = (num_problems * (num_problems + 1)) / 4.0

    #std_wilcoxon = num_problems * (num_problems + 1) * ((2 * num_problems) + 1)
    #std_wilcoxon = math.sqrt(std_wilcoxon / 24.0 - (tie_sum / 48))
    #z_wilcoxon = (w_wilcoxon - mean_wilcoxon) / std_wilcoxon

    #cv_alpha_selected = stats.get_cv_willcoxon(num_problems, alpha)

    #p_value = 2 * stats.get_p_value_normal(z_wilcoxon)

    # if num_problems > 25:


    return r_plus,r_minus


In [8]:
datasets = {'bank':(X_data_bank,targets_bank),
            'adult':(X_data_adult,targets_adult),'nursery':(X_data_nursery,targets_nursery),'bcancer':(X_data_bcancer,targets_bcancer)}

In [9]:
time0 = time.time()
iidresults = experiment1search(data_distribution='iid',datasets=datasets)
print("\n[INFO] Total Time (in minutes) =", (time.time() - time0) / 60)
for result in iidresults:
    print(result)

c:\Users\mario\OneDrive\Escritorio\Programs\gitProjects\TFM_EnsembleFL\.venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\mario\OneDrive\Escritorio\Programs\gitProjects\TFM_EnsembleFL\.venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\mario\OneDrive\Escritorio\Programs\gitProjects\TFM_EnsembleFL\.venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\mario\OneDrive\Escritorio\Programs\gitProjects\TFM_EnsembleFL\.venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\mario\O


[INFO] Total Time (in minutes) = 13.44438531001409
FLGlobal_acc  majority_voting  maj_vot_std  weighted_majority_voting  \
bank                85.034112     0.580415                 85.034112   
adult               80.211439     0.778457                 80.211439   
nursery             90.602023     1.173199                 90.733368   
bcancer             88.436389     4.357873                 89.136890   

FLGlobal_acc  w_m_v_std  
bank           0.580415  
adult          0.778457  
nursery        1.140369  
bcancer        4.099958  
FLlocal_acc  majority_voting  maj_vot_std  weighted_majority_voting  w_m_v_std
bank               84.939951     2.083265                 84.939951   2.083265
adult              80.369048     2.849545                 80.369048   2.849545
nursery            90.354701     3.448114                 90.482906   3.422832
bcancer            88.500000    16.517448                 88.916667  15.707710
centralized_acc       mean       std
bank             85.54421

In [25]:
_,_,_,_,_,_,acc_results,f1_results = iidresults
alpha = 0.05 # Este alpha se podría reducir al 0.01 para ser más estrictos (así tener confianza del 99% en lugar del 95%), pero es un valor bastante usado
columns = list(acc_results.columns)
acc_results2 = acc_results.round(decimals=3).reset_index(drop=True)
f1_results2 = f1_results.round(decimals=3).reset_index(drop=True)
selected_columns = [columns[0], columns[1]]
acc_statistic, acc_p_value, acc_rejected_value, acc_hypothesis = wilconxon(acc_results2[selected_columns])
f1_statistic, f1_p_value, f1_rejected_value, f1_hypothesis = wilconxon(f1_results2[selected_columns])
print("Accuracy results: \n")
print(acc_hypothesis)
print(f"Statistic {acc_statistic}, Rejected Value {acc_rejected_value}, p-value {acc_p_value}")
r_plus,r_minus = wilconxonbest(acc_results2[selected_columns])

print(f"Se tiene que R+={r_plus} y que R-={r_minus}")
if (r_plus>r_minus)&(acc_p_value<alpha): 
    print(f"como R+ > R-, p-valor< alpha: {acc_p_value}< {alpha}, decimos que hay diferencia significativa")
    print(f"Mejor clasificador = {acc_results2.columns[0]}")
elif (r_plus<r_minus)&(acc_p_value<alpha): 
    print(f"como R- > R+, p-valor< alpha: {acc_p_value}< {alpha}, decimos que hay diferencia significativa")
    print(f"Mejor clasificador = {acc_results2.columns[1]}")
    

print("\n F1-score results: \n")
print(f1_hypothesis)
print(f"Statistic {f1_statistic}, Rejected Value {f1_rejected_value}, p-value {f1_p_value}")

r_plus,r_minus = wilconxonbest(f1_results2[selected_columns])

print(f"Se tiene que R+={r_plus} y que R-={r_minus}")
if (r_plus>r_minus)&(f1_p_value<alpha):
    print(f"como R+ > R-, p-valor< alpha: {f1_p_value}< {alpha}, decimos que hay diferencia significativa")
    print(f"Mejor clasificador = {acc_results2.columns[0]}")
elif (r_plus<r_minus)&(f1_p_value<alpha): 
    print(f"como R- > R+, p-valor< alpha: {f1_p_value}< {alpha}, decimos que hay diferencia significativa")
    print(f"Mejor clasificador = {acc_results2.columns[1]}")

Accuracy results: 

Reject H0 with alpha = 0.05 (Different distributions)
Statistic 78.5, Rejected Value 434, p-value 1.7864723034222152e-09


Se tiene que R+=78.5 y que R-=1632.5
como R- > R+, p-valor< alpha: 1.7864723034222152e-09< 0.05, decimos que hay diferencia significativa
Mejor clasificador = weighted_majority_voting

 F1-score results: 

Reject H0 with alpha = 0.05 (Different distributions)
Statistic 70.0, Rejected Value 434, p-value 1.1894465412609634e-09
Se tiene que R+=70.0 y que R-=1641.0
como R- > R+, p-valor< alpha: 1.1894465412609634e-09< 0.05, decimos que hay diferencia significativa
Mejor clasificador = weighted_majority_voting


In [11]:
time0 = time.time()
niid_quantity_skew_res = experiment1search(data_distribution='niid_quantity_skew',datasets=datasets,distribution_param=0.5)
print("\n[INFO] Total Time (in minutes) =", (time.time() - time0) / 60)
for result in niid_quantity_skew_res:
    print(result)

c:\Users\mario\OneDrive\Escritorio\Programs\gitProjects\TFM_EnsembleFL\.venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\mario\OneDrive\Escritorio\Programs\gitProjects\TFM_EnsembleFL\.venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\mario\OneDrive\Escritorio\Programs\gitProjects\TFM_EnsembleFL\.venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\mario\OneDrive\Escritorio\Programs\gitProjects\TFM_EnsembleFL\.venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\mario\O


[INFO] Total Time (in minutes) = 13.562673807144165
FLGlobal_acc  majority_voting  maj_vot_std  weighted_majority_voting  \
bank                84.736689     1.489539                 84.736689   
adult               79.336678     2.252032                 79.336678   
nursery             86.741427     7.894196                 88.266289   
bcancer             86.754218     6.739196                 87.638942   

FLGlobal_acc  w_m_v_std  
bank           1.489539  
adult          2.252032  
nursery        6.443533  
bcancer        6.597265  
FLlocal_acc  majority_voting  maj_vot_std  weighted_majority_voting  w_m_v_std
bank               85.237467     5.657394                 85.237467   5.657394
adult              79.083748     8.061920                 79.083748   8.061920
nursery            85.976345    15.401530                 87.709020  12.874951
bcancer            84.582937    23.558506                 85.449603  23.006854
centralized_acc       mean       std
bank             85.5442

In [26]:
_,_,_,_,_,_,acc_results,f1_results = niid_quantity_skew_res
alpha = 0.05 # Este alpha se podría reducir al 0.01 para ser más estrictos (así tener confianza del 99% en lugar del 95%), pero es un valor bastante usado
columns = list(acc_results.columns)
acc_results2 = acc_results.round(decimals=3).reset_index(drop=True)
f1_results2 = f1_results.round(decimals=3).reset_index(drop=True)
selected_columns = [columns[0], columns[1]]
acc_statistic, acc_p_value, acc_rejected_value, acc_hypothesis = wilconxon(acc_results2[selected_columns])
f1_statistic, f1_p_value, f1_rejected_value, f1_hypothesis = wilconxon(f1_results2[selected_columns])
print("Accuracy results: \n")
print(acc_hypothesis)
print(f"Statistic {acc_statistic}, Rejected Value {acc_rejected_value}, p-value {acc_p_value}")
r_plus,r_minus = wilconxonbest(acc_results2[selected_columns])

print(f"Se tiene que R+={r_plus} y que R-={r_minus}")
if (r_plus>r_minus)&(acc_p_value<alpha): 
    print(f"como R+ > R-, p-valor< alpha: {acc_p_value}< {alpha}, decimos que hay diferencia significativa")
    print(f"Mejor clasificador = {acc_results2.columns[0]}")
elif (r_plus<r_minus)&(acc_p_value<alpha): 
    print(f"como R- > R+, p-valor< alpha: {acc_p_value}< {alpha}, decimos que hay diferencia significativa")
    print(f"Mejor clasificador = {acc_results2.columns[1]}")
    

print("\n F1-score results: \n")
print(f1_hypothesis)
print(f"Statistic {f1_statistic}, Rejected Value {f1_rejected_value}, p-value {f1_p_value}")

r_plus,r_minus = wilconxonbest(f1_results2[selected_columns])

print(f"Se tiene que R+={r_plus} y que R-={r_minus}")
if (r_plus>r_minus)&(f1_p_value<alpha):
    print(f"como R+ > R-, p-valor< alpha: {f1_p_value}< {alpha}, decimos que hay diferencia significativa")
    print(f"Mejor clasificador = {acc_results2.columns[0]}")
elif (r_plus<r_minus)&(f1_p_value<alpha): 
    print(f"como R- > R+, p-valor< alpha: {f1_p_value}< {alpha}, decimos que hay diferencia significativa")
    print(f"Mejor clasificador = {acc_results2.columns[1]}")

Accuracy results: 

Reject H0 with alpha = 0.05 (Different distributions)
Statistic 142.0, Rejected Value 434, p-value 1.2659655101998624e-08


Se tiene que R+=142.0 y que R-=1688.0
como R- > R+, p-valor< alpha: 1.2659655101998624e-08< 0.05, decimos que hay diferencia significativa
Mejor clasificador = weighted_majority_voting

 F1-score results: 

Reject H0 with alpha = 0.05 (Different distributions)
Statistic 92.0, Rejected Value 434, p-value 1.373119840053505e-09
Se tiene que R+=92.0 y que R-=1738.0
como R- > R+, p-valor< alpha: 1.373119840053505e-09< 0.05, decimos que hay diferencia significativa
Mejor clasificador = weighted_majority_voting


In [13]:
time0 = time.time()
niid_dirichlet_label_res = experiment1search(data_distribution='niid_dirichlet_label_skew',datasets=datasets,distribution_param=0.5)
print("\n[INFO] Total Time (in minutes) =", (time.time() - time0) / 60)
for result in niid_dirichlet_label_res:
    print(result)

c:\Users\mario\OneDrive\Escritorio\Programs\gitProjects\TFM_EnsembleFL\.venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\mario\OneDrive\Escritorio\Programs\gitProjects\TFM_EnsembleFL\.venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\mario\OneDrive\Escritorio\Programs\gitProjects\TFM_EnsembleFL\.venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\mario\OneDrive\Escritorio\Programs\gitProjects\TFM_EnsembleFL\.venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\mario\O


[INFO] Total Time (in minutes) = 12.572457869847616
FLGlobal_acc  majority_voting  maj_vot_std  weighted_majority_voting  \
bank                76.679254    21.604648                 76.703045   
adult               71.091046    17.622779                 70.843980   
nursery             72.392490    19.330621                 68.936728   
bcancer             74.055996    16.975885                 70.831509   

FLGlobal_acc  w_m_v_std  
bank          21.617278  
adult         18.127188  
nursery       21.042620  
bcancer       19.465727  
FLlocal_acc  majority_voting  maj_vot_std  weighted_majority_voting  w_m_v_std
bank               88.889677     9.308426                 88.942175   9.361017
adult              86.303084    10.609955                 86.508274  10.686852
nursery            92.701837     5.851081                 91.439915   6.620053
bcancer            94.333995    13.495976                 94.201058  13.197776
centralized_acc       mean       std
bank             85.5442

In [27]:
_,_,_,_,_,_,acc_results,f1_results = niid_dirichlet_label_res
alpha = 0.05 # Este alpha se podría reducir al 0.01 para ser más estrictos (así tener confianza del 99% en lugar del 95%), pero es un valor bastante usado
columns = list(acc_results.columns)
acc_results2 = acc_results.round(decimals=3).reset_index(drop=True)
f1_results2 = f1_results.round(decimals=3).reset_index(drop=True)
selected_columns = [columns[0], columns[1]]
acc_statistic, acc_p_value, acc_rejected_value, acc_hypothesis = wilconxon(acc_results2[selected_columns])
f1_statistic, f1_p_value, f1_rejected_value, f1_hypothesis = wilconxon(f1_results2[selected_columns])
print("Accuracy results: \n")
print(acc_hypothesis)
print(f"Statistic {acc_statistic}, Rejected Value {acc_rejected_value}, p-value {acc_p_value}")
r_plus,r_minus = wilconxonbest(acc_results2[selected_columns])

print(f"Se tiene que R+={r_plus} y que R-={r_minus}")
if (r_plus>r_minus)&(acc_p_value<alpha): 
    print(f"como R+ > R-, p-valor< alpha: {acc_p_value}< {alpha}, decimos que hay diferencia significativa")
    print(f"Mejor clasificador = {acc_results2.columns[0]}")
elif (r_plus<r_minus)&(acc_p_value<alpha): 
    print(f"como R- > R+, p-valor< alpha: {acc_p_value}< {alpha}, decimos que hay diferencia significativa")
    print(f"Mejor clasificador = {acc_results2.columns[1]}")
    

print("\n F1-score results: \n")
print(f1_hypothesis)
print(f"Statistic {f1_statistic}, Rejected Value {f1_rejected_value}, p-value {f1_p_value}")

r_plus,r_minus = wilconxonbest(f1_results2[selected_columns])

print(f"Se tiene que R+={r_plus} y que R-={r_minus}")
if (r_plus>r_minus)&(f1_p_value<alpha):
    print(f"como R+ > R-, p-valor< alpha: {f1_p_value}< {alpha}, decimos que hay diferencia significativa")
    print(f"Mejor clasificador = {acc_results2.columns[0]}")
elif (r_plus<r_minus)&(f1_p_value<alpha): 
    print(f"como R- > R+, p-valor< alpha: {f1_p_value}< {alpha}, decimos que hay diferencia significativa")
    print(f"Mejor clasificador = {acc_results2.columns[1]}")

Accuracy results: 

Reject H0 with alpha = 0.05 (Different distributions)
Statistic 985.0, Rejected Value 434, p-value 7.006990543345637e-10
Se tiene que R+=5120.0 y que R-=985.0
como R+ > R-, p-valor< alpha: 7.006990543345637e-10< 0.05, decimos que hay diferencia significativa
Mejor clasificador = majority_voting

 F1-score results: 

Reject H0 with alpha = 0.05 (Different distributions)
Statistic 1794.0, Rejected Value 434, p-value 0.00011056481625182357
Se tiene que R+=4422.0 y que R-=1794.0
como R+ > R-, p-valor< alpha: 0.00011056481625182357< 0.05, decimos que hay diferencia significativa
Mejor clasificador = majority_voting


In [15]:
dataset = {'nursery':(X_data_nursery,targets_nursery)} #Only nursery bc it is the only one with more than 2 labels
time0 = time.time()
niid_label_quantity_res = experiment1search(data_distribution='niid_label_quantity_skew',datasets=dataset,distribution_param=3)
print("\n[INFO] Total Time (in minutes) =", (time.time() - time0) / 60)
for result in niid_label_quantity_res:
    print(result)

c:\Users\mario\OneDrive\Escritorio\Programs\gitProjects\TFM_EnsembleFL\.venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


c:\Users\mario\OneDrive\Escritorio\Programs\gitProjects\TFM_EnsembleFL\.venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\mario\OneDrive\Escritorio\Programs\gitProjects\TFM_EnsembleFL\.venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\mario\OneDrive\Escritorio\Programs\gitProjects\TFM_EnsembleFL\.venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\mario\OneDrive\Escritorio\Programs\gitProjects\TFM_EnsembleFL\.venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(



[INFO] Total Time (in minutes) = 1.8938461462656657
FLGlobal_acc  majority_voting  maj_vot_std  weighted_majority_voting  \
nursery             55.850652    19.676596                 55.737654   

FLGlobal_acc  w_m_v_std  
nursery       19.683476  
FLlocal_acc  majority_voting  maj_vot_std  weighted_majority_voting  w_m_v_std
nursery            95.764762     4.471573                 95.443366   4.746842
centralized_acc       mean       std
nursery          99.272977  0.275333
FLGlobal_f1  majority_voting  maj_vot_std  weighted_majority_voting  w_m_v_std
nursery            66.294485    17.717373                 66.276682  17.689117
FLlocal_f1  majority_voting  maj_vot_std  weighted_majority_voting  w_m_v_std
nursery           96.330229     4.155233                 96.173293   4.348607
centralized_f1       mean       std
nursery         99.268869  0.278432
               majority_voting  weighted_majority_voting
FL_global_acc                                           
nursery0          

In [28]:
_,_,_,_,_,_,acc_results,f1_results = niid_label_quantity_res
alpha = 0.05 # Este alpha se podría reducir al 0.01 para ser más estrictos (así tener confianza del 99% en lugar del 95%), pero es un valor bastante usado
columns = list(acc_results.columns)
acc_results2 = acc_results.round(decimals=3).reset_index(drop=True)
f1_results2 = f1_results.round(decimals=3).reset_index(drop=True)
selected_columns = [columns[0], columns[1]]
acc_statistic, acc_p_value, acc_rejected_value, acc_hypothesis = wilconxon(acc_results2[selected_columns])
f1_statistic, f1_p_value, f1_rejected_value, f1_hypothesis = wilconxon(f1_results2[selected_columns])
print("Accuracy results: \n")
print(acc_hypothesis)
print(f"Statistic {acc_statistic}, Rejected Value {acc_rejected_value}, p-value {acc_p_value}")
r_plus,r_minus = wilconxonbest(acc_results2[selected_columns])

print(f"Se tiene que R+={r_plus} y que R-={r_minus}")
if (r_plus>r_minus)&(acc_p_value<alpha): 
    print(f"como R+ > R-, p-valor< alpha: {acc_p_value}< {alpha}, decimos que hay diferencia significativa")
    print(f"Mejor clasificador = {acc_results2.columns[0]}")
elif (r_plus<r_minus)&(acc_p_value<alpha): 
    print(f"como R- > R+, p-valor< alpha: {acc_p_value}< {alpha}, decimos que hay diferencia significativa")
    print(f"Mejor clasificador = {acc_results2.columns[1]}")
    

print("\n F1-score results: \n")
print(f1_hypothesis)
print(f"Statistic {f1_statistic}, Rejected Value {f1_rejected_value}, p-value {f1_p_value}")

r_plus,r_minus = wilconxonbest(f1_results2[selected_columns])

print(f"Se tiene que R+={r_plus} y que R-={r_minus}")
if (r_plus>r_minus)&(f1_p_value<alpha):
    print(f"como R+ > R-, p-valor< alpha: {f1_p_value}< {alpha}, decimos que hay diferencia significativa")
    print(f"Mejor clasificador = {acc_results2.columns[0]}")
elif (r_plus<r_minus)&(f1_p_value<alpha): 
    print(f"como R- > R+, p-valor< alpha: {f1_p_value}< {alpha}, decimos que hay diferencia significativa")
    print(f"Mejor clasificador = {acc_results2.columns[1]}")

Accuracy results: 

Reject H0 with alpha = 0.05 (Different distributions)
Statistic 87.0, Rejected Value 137, p-value 0.0027624074665453247
Se tiene que R+=378.0 y que R-=87.0
como R+ > R-, p-valor< alpha: 0.0027624074665453247< 0.05, decimos que hay diferencia significativa
Mejor clasificador = majority_voting

 F1-score results: 

Fail to Reject H0 with alpha = 0.05 (Same distributions)
Statistic 229.5, Rejected Value 137, p-value 0.9507951449265917


Se tiene que R+=235.5 y que R-=229.5


In [18]:
print(np.unique(targets_nursery,return_counts=True))
print(np.unique(targets_adult,return_counts=True))
print(np.unique(targets_bank,return_counts=True)) 

(array([0, 1, 2, 3, 4], dtype=int64), array([4320, 4266,    2, 4044,  328], dtype=int64))
(array([0, 1], dtype=int64), array([24720,  7841], dtype=int64))
(array([0, 1], dtype=int64), array([39922,  5289], dtype=int64))
